In [136]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [137]:
features = pd.read_csv('tmp/features_clean.csv')
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403900 entries, 0 to 403899
Data columns (total 10 columns):
file                403900 non-null object
title_chars         403900 non-null float64
title_words         403900 non-null float64
links               403900 non-null float64
images              403900 non-null float64
paragraphs          403900 non-null float64
chars               403900 non-null float64
words               403900 non-null float64
links_per_words     403900 non-null float64
images_per_words    403900 non-null float64
dtypes: float64(9), object(1)
memory usage: 33.9+ MB


In [138]:
train = pd.read_csv('input/train_v2.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337024 entries, 0 to 337023
Data columns (total 2 columns):
file         337024 non-null object
sponsored    337024 non-null int64
dtypes: int64(1), object(1)
memory usage: 7.7+ MB


In [139]:
test = pd.read_csv('input/sampleSubmission_v2.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66772 entries, 0 to 66771
Data columns (total 2 columns):
file         66772 non-null object
sponsored    66772 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [140]:
tr = pd.merge(train, features, on='file', how='inner')
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336920 entries, 0 to 336919
Data columns (total 11 columns):
file                336920 non-null object
sponsored           336920 non-null int64
title_chars         336920 non-null float64
title_words         336920 non-null float64
links               336920 non-null float64
images              336920 non-null float64
paragraphs          336920 non-null float64
chars               336920 non-null float64
words               336920 non-null float64
links_per_words     336920 non-null float64
images_per_words    336920 non-null float64
dtypes: float64(9), int64(1), object(1)
memory usage: 30.8+ MB


In [141]:
te = pd.merge(test, features, on='file', how='left')
te = te.fillna(0)
te.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66772 entries, 0 to 66771
Data columns (total 11 columns):
file                66772 non-null object
sponsored           66772 non-null int64
title_chars         66772 non-null float64
title_words         66772 non-null float64
links               66772 non-null float64
images              66772 non-null float64
paragraphs          66772 non-null float64
chars               66772 non-null float64
words               66772 non-null float64
links_per_words     66772 non-null float64
images_per_words    66772 non-null float64
dtypes: float64(9), int64(1), object(1)
memory usage: 6.1+ MB


In [142]:
trl = tr.sponsored
trl[:5]

0    0
1    0
2    0
3    0
4    0
Name: sponsored, dtype: int64

In [143]:
del tr['sponsored']
del te['sponsored']
tr.index = tr.file
del tr['file']
te.index = te.file
del te['file']

In [144]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
clf = RandomForestClassifier(n_estimators=100, max_depth=15, oob_score=True, n_jobs=4)

In [145]:
clf.fit(tr, trl)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [146]:
preds = clf.predict_proba(te)

In [147]:
preds = DataFrame(preds)
preds.describe()

,0,1
count,66772.000000,66772.000000
mean,0.902802,0.097198
std,0.073309,0.073309
min,0.000000,0.000000
25%,0.871940,0.051970
50%,0.911105,0.088895
75%,0.948030,0.128060
max,1.000000,1.000000


In [148]:
te.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66772 entries, 119_raw_html.txt to 4059503_raw_html.txt
Data columns (total 9 columns):
title_chars         66772 non-null float64
title_words         66772 non-null float64
links               66772 non-null float64
images              66772 non-null float64
paragraphs          66772 non-null float64
chars               66772 non-null float64
words               66772 non-null float64
links_per_words     66772 non-null float64
images_per_words    66772 non-null float64
dtypes: float64(9)
memory usage: 5.1+ MB


In [149]:
preds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66772 entries, 0 to 66771
Data columns (total 2 columns):
0    66772 non-null float64
1    66772 non-null float64
dtypes: float64(2)
memory usage: 1.5 MB


In [150]:
sub = DataFrame({'file': te.index, 'sponsored': preds[1]})
sub[:5]

,file,sponsored
0,119_raw_html.txt,0.043125
1,167_raw_html.txt,0.172022
2,329_raw_html.txt,0.231335
3,419_raw_html.txt,0.099196
4,431_raw_html.txt,0.171255


In [151]:
sub.to_csv('output/sub_02.csv', index=False)